In [ ]:
#%conda install laspy

Channels:
 - defaults
 - conda-forge
Platform: win-64
Solving environment: ...working... done

## Package Plan ##

  environment location: c:\Users\aylin\anaconda3\envs\segment-geo

  added / updated specs:
    - laspy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    laspy-2.5.4                |  py311h3810d55_1         1.5 MB  conda-forge
    lazrs-python-0.5.2         |  py311h636fa0f_0         372 KB
    numpy-1.26.4               |  py311hdab7c0b_0          11 KB
    numpy-base-1.26.4          |  py311hd01c5d8_0         9.1 MB
    ------------------------------------------------------------
                                           Total:        10.9 MB

The following NEW packages will be INSTALLED:

  laspy              conda-forge/win-64::laspy-2.5.4-py311h3810d55_1 
  lazrs-python       pkgs/main/win-64::lazrs-python-0.5.2-py311h636fa0f_0 

The following packages will be DOWNGRA

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.ndimage import gaussian_filter
from scipy.interpolate import griddata
import laspy
import os


In [ ]:
# Function to read .laz files
def read_las(file_path):
    las = laspy.read(file_path)
    data = {
        "points": np.vstack((las.x, las.y, las.z)).T,
        "intensity": las.intensity,
        "classification": las.classification,
        "return_number": las.return_number,
        "number_of_returns": las.number_of_returns,
    }
    return data

# Function to subsample data
def subsample_data(data, subsample_ratio=0.1):
    points = data["points"]
    sample_size = int(len(points) * subsample_ratio)
    indices = np.random.choice(points.shape[0], sample_size, replace=False)
    return {
        "points": points[indices],
        "intensity": data["intensity"][indices],
        "classification": data["classification"][indices],
        "return_number": data["return_number"][indices],
        "number_of_returns": data["number_of_returns"][indices],
    }



In [1]:

def preprocess_lidar(file_path, voxel_size=1.0):
    """
    Process LiDAR data to generate DSM, DEM, and CHM.
    """
    # Read the LiDAR data
    las = laspy.read(file_path)
    points = np.vstack((las.x, las.y, las.z)).T
    classifications = las.classification

    # Define grid bounds
    x_min, x_max = np.min(points[:, 0]), np.max(points[:, 0])
    y_min, y_max = np.min(points[:, 1]), np.max(points[:, 1])
    grid_width = int((x_max - x_min) / voxel_size) + 1
    grid_height = int((y_max - y_min) / voxel_size) + 1
    grid_x, grid_y = np.meshgrid(
        np.linspace(x_min, x_max, grid_width),
        np.linspace(y_min, y_max, grid_height)
    )

    # DEM: Interpolate ground points
    ground_points = points[classifications == 2]
    dem = griddata(ground_points[:, :2], ground_points[:, 2], (grid_x, grid_y), method="linear")
    dem = gaussian_filter(np.nan_to_num(dem), sigma=2)

    # DSM: Max height grid
    dsm = np.zeros_like(dem)
    for point in points:
        x_idx = int((point[0] - x_min) / voxel_size)
        y_idx = int((point[1] - y_min) / voxel_size)
        dsm[y_idx, x_idx] = max(dsm[y_idx, x_idx], point[2])

    dsm = gaussian_filter(dsm, sigma=2)

    # CHM: DSM - DEM
    chm = dsm - dem
    return grid_x, grid_y, dsm, dem, chm

In [3]:

def calculate_point_density(las_file_path):
    """
    Calculate point density for a given LAS file.

    Args:
        las_file_path (str): Path to the LAS file.

    Returns:
        float: Point density (points per square meter).
    """
    # Load the LAS file
    las = laspy.read(las_file_path)
    
    # Get the number of points
    num_points = len(las.x)
    
    # Area of a 1km x 1km tile
    area = 1000 * 1000  # in square meters
    
    # Calculate point density
    point_density = num_points / area
    return point_density

In [ ]:
# File path to LiDAR data
file_path = "data/Tschernitz/als/als_33470-5716.laz"

# Process the LiDAR file
grid_x, grid_y, dsm, dem, chm = preprocess_lidar(file_path)

# Plot the results
plt.figure(figsize=(18, 6))

# DSM Plot
plt.subplot(1, 3, 1)
plt.contourf(grid_x, grid_y, dsm, cmap="terrain")
plt.title("Digital Surface Model (DSM)")
plt.colorbar(label="Elevation (m)")

# DEM Plot
plt.subplot(1, 3, 2)
plt.contourf(grid_x, grid_y, dem, cmap="terrain")
plt.title("Digital Elevation Model (DEM)")
plt.colorbar(label="Elevation (m)")

# CHM Plot
plt.subplot(1, 3, 3)
plt.contourf(grid_x, grid_y, chm, cmap="viridis")
plt.title("Canopy Height Model (CHM)")
plt.colorbar(label="Height (m)")

plt.tight_layout()
plt.show()


In [ ]:
density = calculate_point_density(file_path)
print(f"Point Density: {density:.2f} points/m²")

In [ ]:
import laspy
import numpy as np
from scipy.interpolate import griddata
import matplotlib.pyplot as plt

def create_dem(file_path, grid_resolution=1.0):
    # Read LAS file
    las = laspy.read(file_path)

    # Extract ground points (class 2 in LAS classification)
    ground_mask = (las.classification == 2)
    ground_points = np.vstack((las.x[ground_mask], las.y[ground_mask], las.z[ground_mask])).T

    # Define grid
    x_min, x_max = np.min(ground_points[:, 0]), np.max(ground_points[:, 0])
    y_min, y_max = np.min(ground_points[:, 1]), np.max(ground_points[:, 1])

    x_grid = np.arange(x_min, x_max, grid_resolution)
    y_grid = np.arange(y_min, y_max, grid_resolution)
    grid_x, grid_y = np.meshgrid(x_grid, y_grid)

    # Interpolate ground points onto the grid
    grid_z = griddata(ground_points[:, :2], ground_points[:, 2], (grid_x, grid_y), method='linear')

    # Fill gaps using nearest neighbor
    grid_z_filled = np.where(np.isnan(grid_z), griddata(ground_points[:, :2], ground_points[:, 2], 
                                                        (grid_x, grid_y), method='nearest'), grid_z)

    return grid_x, grid_y, grid_z_filled



In [ ]:
# File path to LAS/LAZ data
base_dir = "data/Tschernitz/"  # Adjust to your LAS file directory
laz_files = [
    os.path.join(root, file)
    for root, _, files in os.walk(base_dir)
    for file in files
    if file.endswith(".laz")
]

file_path = laz_files[0]
grid_x, grid_y, dem = create_dem(file_path)

plt.figure(figsize=(10, 8))
plt.contourf(grid_x, grid_y, dem, cmap="terrain")
plt.title("Digital Elevation Model (DEM)")
plt.colorbar(label="Elevation (meters)")
plt.show()

In [ ]:
import os
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt

def save_dem_as_images(dem, file_path, output_dir="."):
    """
    Save DEM as both grayscale and colorized images with height bar using the LAS/LAZ filename for naming.
    Creates the output directory if it doesn't exist.
    """
    # Ensure the output directory exists
    os.makedirs(output_dir, exist_ok=True)

    # Extract the base name (without extension) from the file path
    base_name = os.path.splitext(os.path.basename(file_path))[0]

    # Grayscale DEM image
    dem_normalized = (dem - np.nanmin(dem)) / (np.nanmax(dem) - np.nanmin(dem)) * 255
    dem_normalized = np.nan_to_num(dem_normalized).astype(np.uint8)
    grayscale_path = os.path.join(output_dir, f"{base_name}_dem_gray.png")
    Image.fromarray(dem_normalized).save(grayscale_path)
    print(f"Grayscale DEM image saved as {grayscale_path}")

    # Colorized DEM image with height bar
    colorized_path = os.path.join(output_dir, f"{base_name}_dem_colorized.png")
    plt.figure(figsize=(8, 8))
    plt.imshow(dem, cmap="terrain")
    plt.colorbar(label="Height (units)")  # Adjust the label to reflect actual units if known
    plt.title(f"Colorized DEM: {base_name}")
    plt.axis("off")
    plt.savefig(colorized_path, bbox_inches="tight", pad_inches=0.1)
    plt.close()
    print(f"Colorized DEM image saved as {colorized_path}")


save_dem_as_images(dem, file_path, output_dir="output/dem")


In [ ]:
def compute_slope_aspect(dem, grid_resolution):
    """
    Compute slope and aspect from DEM.
    """
    # Gradient in x and y directions
    dz_dx, dz_dy = np.gradient(dem, grid_resolution)

    # Slope in degrees
    slope = np.arctan(np.sqrt(dz_dx**2 + dz_dy**2)) * (180 / np.pi)

    # Aspect in degrees
    aspect = np.arctan2(-dz_dy, dz_dx) * (180 / np.pi)  # Negative dz_dy for azimuth convention
    aspect = (aspect + 360) % 360  # Ensure aspect is in [0, 360]

    return slope, aspect

# Compute slope and aspect
grid_resolution = 1.0  # Example resolution in meters
slope, aspect = compute_slope_aspect(dem, grid_resolution)

# Visualize slope
plt.figure(figsize=(10, 8))
plt.contourf(slope, cmap="viridis")
plt.title("Slope Map")
plt.colorbar(label="Degrees")
plt.show()


In [ ]:
from scipy.ndimage import gaussian_filter

def compute_curvature(dem, grid_resolution):
    """
    Compute curvature from DEM.
    """
    smoothed_dem = gaussian_filter(dem, sigma=1)  # Optional smoothing
    dz_dx, dz_dy = np.gradient(smoothed_dem, grid_resolution)
    d2z_dx2, d2z_dxdy = np.gradient(dz_dx, grid_resolution)
    _, d2z_dy2 = np.gradient(dz_dy, grid_resolution)

    # Curvature formula
    curvature = d2z_dx2 + d2z_dy2
    return curvature

curvature = compute_curvature(dem, grid_resolution)

# Visualize curvature
plt.figure(figsize=(10, 8))
plt.contourf(curvature, cmap="coolwarm")
plt.title("Curvature Map")
plt.colorbar(label="Curvature")
plt.show()
